#### **Question 1**: How do we interpret histogram divisions (bins) in Data Wrangler?

Consider the car dataset we looked at in class

In [1]:
import pandas as pd

carfeatures = pd.read_csv("data/features.csv")

When we look at the "mpg" column in Data Wrangler and inspect the first two bins of the histogram (a "bin" is the interval corresponding to the count in each vertical bar), we see

<img src="figures/mpg1_dw.png" alt="drawing" width="250"/>
<img src="figures/mpg2_dw.png" alt="drawing" width="250"/>

So if a car has exactly 10 mpg, which bin does it get placed in? This is ambiguous, and it could even be repeated and placed in both! (I really doubt the folks who made Data Wrangler would choose to allow data repetition though)

I couldn't find much on Google about how the histogram bins are constructed, but we can deduce what's included in each bin by looking at the frequency table of mpg.

In [2]:
table = pd.crosstab(carfeatures['mpg'], "count")
table

col_0,count
mpg,
9.0,1
10.0,2
11.0,4
12.0,6
13.0,20
...,...
43.4,1
44.0,1
44.3,1


We see that there is:
- 1 car with 9.0 mpg
- 2 cars with 10.0 mpg
- 4 cars with 11.0 mpg
- 6 cars with 12.0 mpg

The only way this could match with what we see in the histogram is if:
- First bin (8-10) includes all cars with $8 < \textrm{mpg} \leq 10$
- Second bin (10-12) includes all cars with $10 < \textrm{mpg} \leq 12$

(You can also check the third bin marked "12-14" and compare it to the frequency table, and conclude that it includes all cars with $12 < \textrm{mpg} \leq 14$)

So Data Wrangler must use histogram bins that are **right-inclusive** a.k.a **left-exclusive** (they include the right endpoint and exclude the left endpoint). This is actually the opposite of the usual convention, i.e. most histogram bins are right-exclusive/left-inclusive.

#### **Question 2**: In Pandas crosstab, can we use multiple columns from a DataFrame (dataset)?

In class, we used pandas.crosstab (but using the nickname "pd" for pandas) to create a frequency table:

In [3]:
table = pd.crosstab(index = carfeatures['cylinders'],columns = "count")
table

col_0,count
cylinders,
3,4
4,204
5,3
6,84
8,103


And then used it to create a more general cross-tabulation (or contingency table):

In [4]:
table_2 = pd.crosstab(index = carfeatures['cylinders'],columns = carfeatures['mpg'])
table_2

mpg,9.0,10.0,11.0,12.0,13.0,14.0,14.5,15.0,15.5,16.0,...,39.4,40.8,40.9,41.5,43.1,43.4,44.0,44.3,44.6,46.6
cylinders,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,4,0,5,...,0,0,0,0,0,0,0,0,0,0
8,1,2,4,6,20,19,1,12,5,8,...,0,0,0,0,0,0,0,0,0,0


In light of the examples I gave on the board, a student naturally asked whether/how we could include multiple columns from the dataset in the cross-tabulation (for example, have the columns include both "mpg" and "acceleration").

We'll look at a smaller dataset instead to demonstrate how to do this. We'll load in a fake dataset containing student data from the two sections of QTM 151.

In [5]:
fake_qtm_data = pd.read_csv("data/fake_QTM_151_data.csv")

You can inspect the dataset using DataWrangler.

We'll make a cross-tabulation of the data of Section vs. Grade Level:

In [6]:
tbl_grade = pd.crosstab(index = fake_qtm_data["Section"], columns = fake_qtm_data["Grade Level"])
tbl_grade

Grade Level,1. (freshman),2. (sophomore),3. (junior),4. (senior)
Section,,,,
Sec. 1,4,4,2,3
Sec. 2,6,8,3,8


And we'll make a cross-tabulation of the data of Section vs. Major:

In [7]:
tbl_major = pd.crosstab(index = fake_qtm_data["Section"], columns = fake_qtm_data["Major"])
tbl_major

Major,bus. adm.,data.sci,math,neuroscience,undeclared
Section,,,,,
Sec. 1,1,4,3,2,3
Sec. 2,6,4,4,5,6


Now, we **can** use both Grade Level and Major as columns in the cross-tabulation, but they way they are arranged is a little bit different than what I wrote on the board:

In [8]:
tbl_2 = pd.crosstab(index = fake_qtm_data["Section"], columns = [fake_qtm_data["Major"], fake_qtm_data["Grade Level"]])
tbl_2

Major           bus. adm.                                         \
Grade Level 1. (freshman) 2. (sophomore) 3. (junior) 4. (senior)   
Section                                                            
Sec. 1                  1              0           0           0   
Sec. 2                  2              1           2           1   

Major            data.sci                                         \
Grade Level 1. (freshman) 2. (sophomore) 3. (junior) 4. (senior)   
Section                                                            
Sec. 1                  1              2           1           0   
Sec. 2                  0              1           0           3   

Major                 math                          neuroscience  \
Grade Level 2. (sophomore) 3. (junior) 4. (senior) 1. (freshman)   
Section                                                            
Sec. 1                   1           1           1             2   
Sec. 2                   1           0           3             2   

Major                                     undeclared                 \
Grade Level 2. (sophomore) 3. (junior) 1. (freshman) 2. (sophomore)   
Section                                                               
Sec. 1                   0           0             0              1   
Sec. 2                   2           1             2              3   

Major                    
Grade Level 4. (senior)  
Section                  
Sec. 1                2  
Sec. 2                1

Above, the index is "Section" so there are two rows of each class section. However, the column headings are **pairs** of the two types of data:
- Column 1 is ('bus. adm.', '1. (freshman')) = # of business administration majors who are freshman.
- Column 2 is ('bus. adm.', '2. (sophomore')) = # of business administration majors who are sophomores.
- Column 3 is # of business adm. majors who are juniors
- Column 4 is # of business adm. majors who are seniors
- Column 5 is # of Data Science majors who are freshman
- ...and so on

The format of the columns is actually much clearer if we explicitly use the "print" function:

In [9]:
print(tbl_2)

Major           bus. adm.                                         \
Grade Level 1. (freshman) 2. (sophomore) 3. (junior) 4. (senior)   
Section                                                            
Sec. 1                  1              0           0           0   
Sec. 2                  2              1           2           1   

Major            data.sci                                         \
Grade Level 1. (freshman) 2. (sophomore) 3. (junior) 4. (senior)   
Section                                                            
Sec. 1                  1              2           1           0   
Sec. 2                  0              1           0           3   

Major                 math                          neuroscience  \
Grade Level 2. (sophomore) 3. (junior) 4. (senior) 1. (freshman)   
Section                                                            
Sec. 1                   1           1           1             2   
Sec. 2                   1           0        

By default, pandas.crosstab just omits columns that have only zeros. Since there are no freshman math majors, that column does not appear. (If you add the optional argument "dropna = False" in pd.crosstab it will keep these columns)

What if we would like to arrange the data so that instead of "pairs" attributes, the first five columns are the majors, and the last four columns are the grade level?

What we need to do is combine the tables "tbl_grade" and "tbl_major" into a single table. I.e. stack them "side-by-side". In data and computer science, the word **concatenate** is usually used instead of "combine" or "stack".

In [10]:
# pd.concat is a function for "concatenating" DataFrames

# if we do axis = 0, it stacks them on top of each other (not what we're trying to do)
# axis = 1 it stacks them side-by-side (what we want)

tbl_major_grade = pd.concat([tbl_major, tbl_grade], axis = 1)
print(tbl_major_grade)

         bus. adm.  data.sci  math  neuroscience  undeclared  1. (freshman)  \
Section                                                                       
Sec. 1           1         4     3             2           3              4   
Sec. 2           6         4     4             5           6              6   

         2. (sophomore)  3. (junior)  4. (senior)  
Section                                            
Sec. 1                4            2            3  
Sec. 2                8            3            8  


Can arrange them by grade level first too:

In [11]:
tbl_grade_major = pd.concat([tbl_grade, tbl_major], axis = 1)
print(tbl_grade_major)

         1. (freshman)  2. (sophomore)  3. (junior)  4. (senior)  bus. adm.  \
Section                                                                       
Sec. 1               4               4            2            3          1   
Sec. 2               6               8            3            8          6   

         data.sci  math  neuroscience  undeclared  
Section                                            
Sec. 1          4     3             2           3  
Sec. 2          4     4             5           6  
